# Import wastewater data from Ville de Québecinto a data-model ready sheet

In [1]:
import datetime
import sqlalchemy
import pandas as pd
import sys
sys.path.insert(1, '/workspaces/ODM Import/')
from wbe_odm.odm_mappers import modeleau_mapper
from wbe_odm import odm

1. Enter the path to the file containing the data you want to import.
2. Enter the sheet name containing the raw data.

In [2]:
filepath = "/workspaces/ODM Import/Data/Lab/modelEAU/COVIDProject_Lab Measurements.xlsx"
sheet_name = "Lab analyses"
mapper = modeleau_mapper.ModelEauMapper()
mapper.read(filepath, sheet_name)
odm_instance = odm.Odm()
odm_instance.load_from(mapper)

today = datetime.datetime.today().strftime("%Y-%m-%d")

odm_instance.to_csv("/workspaces/ODM Import/Data/Lab/modelEAU/script_results", file_prefix=f"modelEAULab_{today}")

## Stop here if you are using this sheet to get excel sheets from your lab data.

Continue if you want to enter the lab data into a database

In [5]:
import odm
#preparing for DB:
measurements = measurements.rename(columns={"assayMethodID": "assayID"})
for col in ["index", "reporterID"]:
    if col in samples.columns:
        samples.drop([col],axis=1, inplace=True)
if "WwMeasureID" not in measurements.columns:
    measurements.insert(loc=0, column="WwMeasureID", value=None)

path = "Data/WBE.db"
engine = sqlalchemy.create_engine(f"sqlite:///{path}")

#Preparing for testing the "Replace" method"
engine.execute("delete from WWMeasure")
engine.execute("delete from Sample")

table_name = "WWMeasure"
odm.replace_into_db(measurements, table_name, engine)
table_name = "Sample"
odm.replace_into_db(samples, table_name, engine)